In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import warnings
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import string
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    return soup


def extract_links(url):
    soup = get_soup(url)
    aitems = []
    for h3 in soup.find_all("h3"):
        try:
            a = h3.a["href"]
        except:
            KeyError
        aitems.append(a)
    return aitems


all_links1 = [
    f"https://xwatch.vn/dong-ho-nam-pc85-page{idx_page}.html"
    for idx_page in range(1, 2)
]
all_links2 = [
    f"https://xwatch.vn/dong-ho-nu-pc86-page{idx_page}.html"
    for idx_page in range(1, 2)
]

all_links = []

with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
    futures = [executor.submit(extract_links, url) for url in all_links1 + all_links2]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

all_links = [link for sublist in results for link in sublist]

In [3]:
all_links = list(set(all_links))

all_links

['https://xwatch.vn/dong-ho-nu/og3811dlk-t-p1915.html',
 'https://xwatch.vn/dong-ho-nu/og386lsk-t-p1907.html',
 'https://xwatch.vn/dong-ho-nu/em0674-81a-p953596.html',
 'https://xwatch.vn/dong-ho-nu/op68322sk-t-p2946.html',
 'https://xwatch.vn/dong-ho-nu/ra-nr2006a10b-p958004.html',
 'https://xwatch.vn/dong-ho-nu/op68322dw-t-hx-p957781.html',
 'https://xwatch.vn/dong-ho-nam/op99141-71ags-t-cv-p957355.html',
 'https://xwatch.vn/dong-ho-nam/ra-as0101s10b-p957858.html',
 'https://xwatch.vn/dong-ho-nu/op2467lk-t-p2908.html',
 'https://xwatch.vn/dong-ho-nu/em0530-81d-p931269.html',
 'https://xwatch.vn/dong-ho-nam/ra-ac0f11l10b-p958465.html',
 'https://xwatch.vn/dong-ho-nam/op995-9agsk-t-p32460.html',
 'https://xwatch.vn/dong-ho-nam/sg10711202te-p957510.html',
 'https://xwatch.vn/dong-ho-nu/ew3254-87a-p953489.html',
 'https://xwatch.vn/dong-ho-nam/sg800511102cf-p957855.html',
 'https://xwatch.vn/dong-ho-nam/op5695msk-t-p958108.html',
 'https://xwatch.vn/dong-ho-nam/og35861amr-gl-p957506.html

In [4]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup


soup = get_soup(all_links[0])
details = {}
nameAndtype = soup.find("div", class_="product_name")
ds = nameAndtype.getText().lower().replace("chính hãng","").strip().split(" ")
type = ds[-1].strip().upper()
price = (
    soup.find_all("h3", class_="price_current")[0]
    .text.replace("₫", "")
    .strip()
)

content = soup.find("div", class_="table-condensed compare_table")
rows = content.find_all("tr")
for row in rows:
    text = row.getText().replace("\t", "").replace("\n", "").replace("\r", "")
    if len(text.split(":")) == 2:
        key, value = text.split(":")
        details[key.lower().strip()] = value.strip()
a = {"Giá tiền": price, "Mã sản phẩm": type, **details}
print(a)

{'Giá tiền': '13.530.000', 'Mã sản phẩm': 'OG3811DLK-T', 'nhãn hiệu': 'Ogival', 'nguồn gốc': 'Thụy Sỹ', 'kiểu máy': 'Swiss quartz (chạy pin)', 'kích cỡ': '30mm', 'chất liệu vỏ': 'Thép không gỉ/Mạ PVD', 'chất liệu dây': 'Thép không gỉ/Mạ PVD', 'chất liệu kính': 'Kính Sapphire', 'độ chịu nước': '30M', 'chức năng khác': 'Đính đá', 'bảo hiểm': '5 năm cả lỗi người dùng tại Xwatch', 'bảo hành quốc tế': '2 năm', 'giới tính': 'Nữ'}


In [5]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup
def get_product_info(url):
    soup = get_soup(url)
    details = {}
    nameAndtype = soup.find("div", class_="product_name")
    ds = nameAndtype.getText().lower().replace("chính hãng","").strip().split(" ")
    type = ds[-1].strip().upper()
    price = (
        soup.find_all("h3", class_="price_current")[0]
        .text.replace("₫", "")
        .strip()
    )

    content = soup.find("div", class_="table-condensed compare_table")
    rows = content.find_all("tr")
    for row in rows:
        text = row.getText().replace("\t", "").replace("\n", "").replace("\r", "")
        if len(text.split(":")) == 2:
            key, value = text.split(":")
            details[key.strip()] = value.strip()
    a = {"Giá tiền": price, "Mã sản phẩm": type, **details}
    print(a)
    return {"Giá tiền": price, "Mã sản phẩm": type, **details}

In [6]:
def crawl_data_from_web(all_links):
    data_frame_watchs = pd.DataFrame()
    with ThreadPoolExecutor(max_workers=25) as executor:
        results = executor.map(get_product_info, all_links)
        for result in results:
            data_frame_watchs = data_frame_watchs.append(result, ignore_index=True)
    return data_frame_watchs

data_frame_watchs = crawl_data_from_web(all_links)

data_frame_watchs

{'Giá tiền': '8.785.000', 'Mã sản phẩm': 'EM0530-81D', 'Nhãn hiệu': 'Citizen', 'Nguồn gốc': 'Nhật Bản', 'Kiểu máy': 'Eco-drive', 'Kích cỡ': '28,7mm', 'Chất liệu vỏ': 'Thép không gỉ', 'Chất liệu dây': 'Thép không gỉ', 'Chất liệu kính': 'Kính Sapphire', 'Độ chịu nước': '50M', 'Bảo hiểm': '5 năm cả lỗi người dùng tại Xwatch', 'Bảo hành quốc tế': '1 năm tiêu chuẩn đồng hồ chính hãng', 'Giới tính': 'Nữ'}
{'Giá tiền': '28.280.000', 'Mã sản phẩm': 'OG358.53A42GR-GL', 'Nhãn hiệu': 'Ogival', 'Nguồn gốc': 'Thụy Sỹ', 'Kiểu máy': 'Automatic', 'Kích cỡ': '42mm', 'Chất liệu vỏ': 'Thép không gỉ/Mạ PVD', 'Chất liệu dây': 'Dây da', 'Chất liệu kính': 'Kính Sapphire', 'Độ chịu nước': '30M', 'Chức năng khác': 'Skeleton (Lộ máy toàn bộ)', 'Bảo hiểm': '5 năm cả lỗi người dùng tại Xwatch', 'Bảo hành quốc tế': '2 năm', 'Giới tính': 'Nam'}
{'Giá tiền': '13.530.000', 'Mã sản phẩm': 'OG3811DLK-T', 'Nhãn hiệu': 'Ogival', 'Nguồn gốc': 'Thụy Sỹ', 'Kiểu máy': 'Swiss quartz (chạy pin)', 'Kích cỡ': '30mm', 'Chất liệu 

,Giá tiền,Mã sản phẩm,Nhãn hiệu,Nguồn gốc,Kiểu máy,Kích cỡ,Chất liệu vỏ,Chất liệu dây,Chất liệu kính,Độ chịu nước,Chức năng khác,Bảo hiểm,Bảo hành quốc tế,Giới tính,Độ dày
0,13.530.000,OG3811DLK-T,Ogival,Thụy Sỹ,Swiss quartz (chạy pin),30mm,Thép không gỉ/Mạ PVD,Thép không gỉ/Mạ PVD,Kính Sapphire,30M,Đính đá,5 năm cả lỗi người dùng tại Xwatch,2 năm,Nữ,NaN
1,11.820.000,OG386LSK-T,Ogival,Thụy Sỹ,Swiss quartz (Máy pin),21x29mm,Thép không gỉ/Mạ PVD,Thép không gỉ/Mạ PVD,Sapphire,30M,NaN,5 năm cả lỗi người dùng tại Xwatch,2 năm,Nữ,NaN
2,9.585.000,EM0674-81A,NaN,Nhật Bản,Eco- Drive,31 mm,Thép - PVD vàng,Thép - PVD vàng,Kính Sapphire,50M,NaN,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nữ,NaN
3,2.850.000,OP68322SK-T,Olym Pianus,Nhật Bản,Quartz (Máy pin),24 mm,Thép không gỉ 316L/Mạ vàng PVD,Thép không gỉ 316L / Mạ vàng PVD,Kính Sapphire,30 m,Lịch ngày,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nữ,NaN
4,8.960.000,RA-NR2006A10B,Orient,Nhật Bản,Automatic (Máy cơ),32mm,Thép không gỉ 316L,Thép không gỉ,Kính cứng,50 m,Lịch ngày,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nữ,11.1mm
5,3.980.000,OP68322DW-T-HX,Olym Pianus,Nhật Bản,Japan movement (Máy pin),25mm,Thép không gỉ/Mạ PVD,Thép không gỉ/Mạ PVD,Kính Sapphire,30m,Lịch ngày,5 năm cả lỗi người dùng tại Xwatch,2 năm,Nữ,7mm
6,7.100.000,OP99141-71AGS-T-CV,Olym Pianus,Nhật Bản,Automatic,40mm,Thép không gỉ 316L/Mạ PVD,Thép không gỉ 316L/Mạ PVD,Kính Sapphire,30m,NaN,5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,NaN
7,12.850.000,RA-AS0101S10B,Orient,Nhật Bản,Automatic,41.5 mm,Thép không gỉ 316L,Thép không gỉ 316L/ Mạ vàng PVD,Kính cứng/ Cong,30M,"Open heart, lộ đáy, lịch Sun and Moon, lên cót...",5 năm cả lỗi người dùng tại Xwatch,1 năm,Nam,13.8mm
8,3.280.000,OP2467LK-T,Olym Pianus,Nhật Bản,Quartz (Máy pin),26 mm,Thép không gỉ 316L / Mạ vàng PVD,Thép không gỉ 316L / Mạ vàng PVD,Kính Sapphire,30 m,Lịch ngày,5 năm cả lỗi người dùng tại Xwatch,2 năm,Nữ,5mm
9,8.785.000,EM0530-81D,Citizen,Nhật Bản,Eco-drive,"28,7mm",Thép không gỉ,Thép không gỉ,Kính Sapphire,50M,NaN,5 năm cả lỗi người dùng tại Xwatch,1 năm tiêu chuẩn đồng hồ chính hãng,Nữ,NaN
